# inspect menor simulation


Kerchunk demos:

- https://fsspec.github.io/kerchunk/tutorial.html
- https://nbviewer.org/gist/rsignell-usgs/ef435a53ac530a2843ce7e1d59f96e22
- https://medium.com/pangeo/using-kerchunk-with-uncompressed-netcdf-64-bit-offset-files-cloud-optimized-access-to-hycom-ocean-9008ba6d0d67
- https://nbviewer.org/gist/rsignell/932ead1fbf143061bd620e7cf764cbb0


In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr

%matplotlib inline
import matplotlib.pyplot as plt

import pynsitu as pin
crs = pin.maps.crs

#import gstools as gs
#from xhistogram.xarray import histogram

# kerchunk
from fsspec.implementations.local import LocalFileSystem
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
from dask import delayed

ERROR 1: PROJ: proj_create_from_database: Open of /home1/datawork/aponte/miniconda3/envs/equinox/share/proj failed


In [14]:
local = True
#local = False

if local:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()
    client = Client(cluster)
else:
    from dask.distributed import Client, LocalCluster
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster()
    cluster = PBSCluster(cores=14, processes=14)
    #cluster = PBSCluster(cores=14, processes=7)
    w = cluster.scale(jobs=10)
    #
    client = Client(cluster)

/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-message

In [ ]:
fs = LocalFileSystem()

In [3]:
menor_dir = "/dataref/ref3/public/modeles_marc/f2_1200_sn/best_estimate/"
#flist = fs.glob(menor_dir+"*/*Z.nc")
flist = fs.glob(menor_dir+"2023/*Z.nc")

In [38]:
ds = xr.open_dataset(flist[0])
ds

<xarray.Dataset>
Dimensions:      (ni: 1101, nj: 463, ni_u: 1101, nj_u: 463, ni_v: 1101,
                  nj_v: 463, ni_f: 1101, nj_f: 463, time: 1, level: 60)
Coordinates: (12/16)
  * ni           (ni) float32 0.0 1.0 2.0 3.0 ... 1.098e+03 1.099e+03 1.1e+03
  * nj           (nj) float32 0.0 1.0 2.0 3.0 4.0 ... 459.0 460.0 461.0 462.0
  * ni_u         (ni_u) float32 0.5 1.5 2.5 3.5 ... 1.098e+03 1.1e+03 1.1e+03
  * nj_u         (nj_u) float32 0.0 1.0 2.0 3.0 4.0 ... 459.0 460.0 461.0 462.0
  * ni_v         (ni_v) float32 0.0 1.0 2.0 3.0 ... 1.098e+03 1.099e+03 1.1e+03
  * nj_v         (nj_v) float32 0.5 1.5 2.5 3.5 4.5 ... 459.5 460.5 461.5 462.5
    ...           ...
  * time         (time) datetime64[ns] 2023-01-01
  * level        (level) float32 -0.9917 -0.975 -0.9583 ... -0.025 -0.008333
    longitude_u  (nj_u, ni_u) float32 ...
    longitude_v  (nj_v, ni_v) float32 ...
    latitude_u   (nj_u, ni_u) float32 ...
    latitude_v   (nj_v, ni_v) float32 ...
Data variables: (12/15)
    SIG          (level) float32 ...
    Csu_sig      (level) float32 ...
    hc           (nj, ni) float32 ...
    b            float32 ...
    theta        float32 ...
    H0           (nj, ni) float32 ...
    ...           ...
    UZ           (time, level, nj_u, ni_u) float32 ...
    VZ           (time, level, nj_v, ni_v) float32 ...
    SAL          (time, level, nj, ni) float32 ...
    TEMP         (time, level, nj, ni) float32 ...
    TAUX         (time, nj_u, ni_u) float32 ...
    TAUY         (time, nj_v, ni_v) float32 ...
Attributes: (12/55)
    limi:                    0
    lima:                    1100
    pasi:                    1
    ljmi:                    0
    ljma:                    462
    pasj:                    1
    ...                      ...
    quality_index:           0
    product_name:            MARC_F2-MARS3D-MENOR1200_20230101T0000Z.nc
    comment:                 Use of Meteo-France ARPEGEHR meteorological data
    start_date:              2023-01-01T00:00:00Z
    stop_date:               2023-01-01T00:00:00Z
    field_type:              3-hourly

In [9]:
%time
d0 = SingleHdf5ToZarr(flist[0]).translate()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.3 µs


/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py:541: UserWarning: The following excepion was caught and quashed while traversing HDF5
argument of type 'Empty' is not iterable
Traceback (most recent call last):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py", line 354, in _translator
    elif _is_netcdf_datetime(h5obj) or _is_netcdf_variable(h5obj):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py", line 670, in _is_netcdf_datetime
    return units and "since" in units
TypeError: argument of type 'Empty' is not iterable

  warnings.warn(msg)


In [11]:
d0

{'version': 1,
 'refs': {'.zgroup': '{"zarr_format":2}',
  '.zattrs': '{"Conventions":"CF-1.6 OCO-1.3.1 COMODO-1.0","arakawa_grid_type":"C1","area":"North Western Mediterranean Sea","comment":"Use of Meteo-France ARPEGEHR meteorological data","contact":"cdoco-exploit@ifremer.fr","creation_date":"2023-01-03T17:34:41Z","data_centre":"IFREMER OCO DATA CENTER","data_centre_references":"http:\\/\\/www.marc.ifremer.fr\\/","data_type":"OCO oriented grid","depth_resolution":"n\\/a","distribution_statement":"Data restrictions: for registered users only","easternmost_longitude":"15.9999","easting":"longitude","field_type":"3-hourly","forecast_range":"4-days forecast","forecast_type":"forecast","format_version":"1.3.1","global_imax":1100,"global_imin":0,"global_jmax":462,"global_jmin":0,"grid_projection":"n\\/a","history":"2023-01-03T17:34:41Z: creation","institution":"IFREMER","institution_references":"http:\\/\\/www.ifremer.fr\\/","latitude_resolution":"1.082250000000000E-002","lima":1100,"limi

### generate indivual json

In [15]:
from pathlib import Path
import os
import ujson

In [20]:
json_dir = "/home/datawork-lops-osi/aponte/swot/cswot/menor/"

def gen_json(u):
    so = {}
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        p = u.split('/')
        date = p[-2]
        fname = p[-1]
        outf = f'{json_dir}{date}.{fname}.json'
        print(outf)
        with open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

In [21]:
gen_json(flist[0])

/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T0000Z.nc.json


/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py:541: UserWarning: The following excepion was caught and quashed while traversing HDF5
argument of type 'Empty' is not iterable
Traceback (most recent call last):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py", line 354, in _translator
    elif _is_netcdf_datetime(h5obj) or _is_netcdf_variable(h5obj):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py", line 670, in _is_netcdf_datetime
    return units and "since" in units
TypeError: argument of type 'Empty' is not iterable

  warnings.warn(msg)


In [26]:
gen_json_delayed = delayed(gen_json)

In [67]:
#client.compute([gen_json_delayed(u) for u in flist]); # only process 56 files

In [ ]:
#client.map(gen_json_delayed, flist)
futures = client.map(gen_json, flist)

/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230308T2100Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230307T1800Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230308T1800Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230309T0000Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230309T0600Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230309T0300Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230314T0300Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230310T1800Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230314T0600Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230314T1200

### combine json

In [76]:
json_files = sorted(glob(os.path.join(json_dir, "*.nc.json")))
print(len(json_files))
json_files[:10]

2920


['/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T0000Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T0300Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T0600Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T0900Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T1200Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T1500Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T1800Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T2100Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230102T0000Z.nc.json',
 '/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.

In [77]:
mzz = MultiZarrToZarr(
    json_files,
    concat_dims = "time",
    identical_dims = ['level', 'ni', 'nj', 'ni_u', 'nj_u', 'ni_v', 'nj_v',],
)

In [78]:
d = mzz.translate()

json_main = os.path.join(json_dir,'combined.json')
with open(json_main, 'wb') as f:
    f.write(ujson.dumps(d).encode())

In [80]:
#ujson.load

In [81]:
%%time
backend_args = {"consolidated": False, "storage_options": {"fo": d,}}
#xr.open_dataset(d, engine="kerchunk", backend_kwargs=backend_args)
ds = xr.open_dataset(d, engine="kerchunk")

CPU times: user 200 ms, sys: 8 ms, total: 208 ms
Wall time: 201 ms


In [82]:
ds

<xarray.Dataset>
Dimensions:      (time: 2920, nj: 463, ni: 1101, nj_u: 463, ni_u: 1101,
                  nj_v: 463, ni_v: 1101, level: 60, ni_f: 1101, nj_f: 463)
Coordinates: (12/16)
    latitude     (time, nj, ni) float64 ...
    latitude_u   (time, nj_u, ni_u) float32 ...
    latitude_v   (time, nj_v, ni_v) float32 ...
  * level        (level) float32 -0.9917 -0.975 -0.9583 ... -0.025 -0.008333
    longitude    (time, nj, ni) float64 ...
    longitude_u  (time, nj_u, ni_u) float32 ...
    ...           ...
  * ni_v         (ni_v) float32 0.0 1.0 2.0 3.0 ... 1.098e+03 1.099e+03 1.1e+03
  * nj           (nj) float32 0.0 1.0 2.0 3.0 4.0 ... 459.0 460.0 461.0 462.0
  * nj_f         (nj_f) float32 0.5 1.5 2.5 3.5 4.5 ... 459.5 460.5 461.5 462.5
  * nj_u         (nj_u) float32 0.0 1.0 2.0 3.0 4.0 ... 459.0 460.0 461.0 462.0
  * nj_v         (nj_v) float32 0.5 1.5 2.5 3.5 4.5 ... 459.5 460.5 461.5 462.5
  * time         (time) datetime64[ns] 2023-01-01 ... 2023-12-31T21:00:00
Data variables:
    H0           (time, nj, ni) float32 ...
    HX           (time, nj_u, ni_u) float32 ...
    HY           (time, nj_v, ni_v) float32 ...
    SAL          (time, level, nj, ni) float32 ...
    TAUX         (time, nj_u, ni_u) float32 ...
    TAUY         (time, nj_v, ni_v) float32 ...
    TEMP         (time, level, nj, ni) float32 ...
    UZ           (time, level, nj_u, ni_u) float32 ...
    VZ           (time, level, nj_v, ni_v) float32 ...
    XE           (time, nj, ni) float32 ...
    hc           (time, nj, ni) float32 ...
Attributes: (12/55)
    Conventions:             CF-1.6 OCO-1.3.1 COMODO-1.0
    arakawa_grid_type:       C1
    area:                    North Western Mediterranean Sea
    comment:                 Use of Meteo-France ARPEGEHR meteorological data
    contact:                 cdoco-exploit@ifremer.fr
    creation_date:           2023-01-03T17:34:41Z
    ...                      ...
    source:                  MARS3D V10.10
    southernmost_latitude:   39.5000
    start_date:              2023-01-01T00:00:00Z
    stop_date:               2023-01-01T00:00:00Z
    title:                   MARC MENOR 1200 forecast
    westernmost_longitude:   0.0000

ERROR! Session/line number was not unique in database. History logging moved to new session 855


/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py:541: UserWarning: The following excepion was caught and quashed while traversing HDF5
argument of type 'Empty' is not iterable
Traceback (most recent call last):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py", line 354, in _translator
    elif _is_netcdf_datetime(h5obj) or _is_netcdf_variable(h5obj):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py", line 670, in _is_netcdf_datetime
    return units and "since" in units
TypeError: argument of type 'Empty' is not iterable

  warnings.warn(msg)
/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/kerchunk/hdf.py:541: UserWarning: The following excepion was caught and quashed while traversing HDF5
argument of type 'Empty' is not iterable
Traceback (most recent call last):
  File "/home1/datawork/aponte/miniconda3/envs/equi

/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T1800Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230101T2100Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230106T0300Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230106T0000Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230106T2100Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230107T0000Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230109T0900Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230106T1800Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230107T0300Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20230109T1200

In [81]:
#ds["UZ"].isel(level=-1).plot(col="time", col_wrap)
ds["UZ"].isel(level=-1).mean("time").plot()

Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
distributed.nanny - ERROR - Worker process died unexpectedly
distributed.nanny - ERROR - Worker process died unexpectedly
distributed.nanny - ERROR - Worker process died unexpectedly
distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
distributed.nanny - ERROR - Worker process died unexpectedly
distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home1/datawork/aponte/miniconda3/envs/equinox/lib/python3.10/site-packages/distributed/compatibility.py", line 236, in asyncio_run
    return loop.run_until_complete(main)
  File "/home1/datawork/aponte/miniconda3/envs/

/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231107T2100Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231109T1500Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231112T0000Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231112T0300Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231114T0600Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231115T0300Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231117T0000Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231117T1200Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231116T0600Z.nc.json
/home/datawork-lops-osi/aponte/swot/cswot/menor/2023.MARC_F2-MARS3D-MENOR1200_20231118T1200

In [57]:
len(flist)

2920

In [59]:
24*365/3

2920.0